# ESP EV

This notebook defines an agent to play the ESP card game. It runs on Python 3.8.

### Contents:
1. Imports
1. Core
    1. Agent Base Class
    1. Helper Functions
1. Specific Agents
    1. Greedy Agent
    1. Optimal Agent
    1. Information Agent
1. Sweeps
    1. Information Agent vs $\gamma$
    2. Greedy, Optimal, Info Agent EV vs Initial Deck

## Imports

In [1]:
from __future__ import absolute_import, division, print_function
import os, sys, pdb, pickle
from multiprocessing import Pool
from fractions import Fraction
import time, math
from copy import deepcopy
from collections import OrderedDict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Agent Base Class

Use this as the base class for other agents, such as GreedyAgent.

In [2]:
class Agent(object):
    def __init__(self, initial=None):
        if initial: self.reset(initial)
    
    def reset(self, initial):
        self.initial = initial
        self.group_num = len(initial)
        self.size = sum(self.initial)
        self.states = {self.initial:Fraction(1)}
        self.correct = 0
        self.group_prob = [ Fraction(x, sum(self.initial)) for x in self.initial ]
    
    def set_composition(self, composition):
        example_element = list(composition.keys())[0]
        self.group_num = len(example_element)
        self.size = sum(example_element)
        self.states = composition
        self.correct = 0
        self.update_group_prob()
        
    def serialize(self):
        return 'x'.join(map(str, self.initial)) + ' ' + ' '.join(sorted(['|'.join(map(str, k)) + ':%s'%str(v) for k,v in self.states.items()]))
    
    def get_state_entropy(self, total=False):
        entropy = 0.0
        for k,v in self.states.items():
            micro_per_macro = math.factorial(sum(k)) / np.prod(map(math.factorial, k)) if total else 1.0
            entropy -= v * np.log2(v / micro_per_macro)
        return entropy
    
    def make_guess(self):
        raise NotImplementedError('This method must be implemented.')
    
    def ev(self, memo=None):
        if self.size <= 0: return 0
        if memo is None: memo = {}
        serial = self.serialize()
        if serial in memo: return memo[serial]
        guess = self.make_guess()
        
        copy0 = deepcopy(self)
        if copy0.update_state(guess, False): ev0 = copy0.ev(memo)
        else: ev0 = 0
        copy1 = deepcopy(self)
        if copy1.update_state(guess,  True): ev1 = copy1.ev(memo) + 1
        else: ev1 = 0
        
        ev = (1 - self.group_prob[guess]) * ev0 + self.group_prob[guess] * ev1
        memo[serial] = ev
        return ev
    
    def update_group_prob(self):
        if self.size:
            self.group_prob = [0] * self.group_num
            for k,v in self.states.items():
                for i in range(len(k)):
                    self.group_prob[i] += v * k[i] / self.size
            if sum(self.group_prob) != 1:
                print('Warning: sum of group probabilities %f != 1.0'%sum(self.group_prob))
    
    def update_state(self, guess, correct):
        self.correct += correct
        self.size -= 1
        next_states = {}
        for k,v in self.states.items():
            if correct and k[guess]:
                bayes = Fraction(k[guess], sum(k)) * v / self.group_prob[guess]
                kl = list(k)
                kl[guess] -= 1
                kl = tuple(kl)
                if kl not in next_states: next_states[kl] = Fraction(0)
                next_states[kl] += bayes
            if not correct and sum(k) - k[guess]:
                bayes0 = (1 - Fraction(k[guess], sum(k))) * v / (1 - self.group_prob[guess])
                for other in range(self.group_num):
                    if other == guess or not k[other]: continue
                    bayes = bayes0 * Fraction(k[other], sum(k) - k[guess])
                    kl = list(k)
                    kl[other] -= 1
                    kl = tuple(kl)
                    if kl not in next_states: next_states[kl] = Fraction(0)
                    next_states[kl] += bayes
        self.states = next_states
        if not len(self.states): return False
        if sum(self.states.values()) != 1:
            print('Warning: sum of state probabilities %f != 1.0'%sum(self.states.values()))
        self.update_group_prob()
        return True
    
    def run(self, deck):
        self.reset(tuple(np.bincount(deck)))
        for card in deck:
            guess = self.make_guess()
            self.update_state(guess, guess==card)
        return self.correct

## Helper Functions

1. generate_all_decks: given a compressed representation of the composition of cards, generates a list of all decks.
2. deck_ev: computes the expected number of correct guesses for a given agent, given a compressed representation of the composition of cards.

In [3]:
def generate_all_decks(initial):
    all_decks = []
    if sum(initial) == 1:
        for i in range(len(initial)):
            if initial[i] > 0: return [[i]]
    for i in range(len(initial)):
        if initial[i] > 0:
            initial[i] -= 1
            recur_decks = generate_all_decks(initial)
            initial[i] += 1
            all_decks += [[i] + deck for deck in recur_decks]
    return all_decks
all_decks = generate_all_decks([3,3,3])
print('Number of Unique Decks:', len(all_decks))

from functools import partial
def f(l, a):
    return a.run(l)

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

def deck_ev(agent, initial, verbose=True, **kwargs):
    nproc = 8
    p = Pool(nproc)
    batch_size = 2 * nproc
    a = agent(**kwargs)
    
    all_decks = generate_all_decks(list(initial))
    correct_guesses = []
    for idx, decks in enumerate(chunks(all_decks, batch_size)):
        correct_guesses += p.map(partial(f, a=a), decks)
        if verbose:
            print('\rOn deck %s (%04d/%04d) | EV %.6f'%(''.join(map(str, decks[-1])), min((idx+1)*batch_size, len(all_decks)), len(all_decks), np.mean(correct_guesses)), end='')
            sys.stdout.flush()
    if verbose:
        print()
    p.terminate()
    return np.mean(correct_guesses)

def random_argmax(rng, scores: np.ndarray):
    probs = (scores==scores.max()).astype(np.float32)
    probs /= probs.sum()
    return rng.choice(np.arange(scores.size), p=probs)

Number of Unique Decks: 1680


## Greedy Agent

Selects the card maximizing unconditioned-probability over all states.

In [4]:
class GreedyAgent(Agent):
    def __init__(self, **kwargs):
        super(GreedyAgent, self).__init__(**kwargs)
    def reset(self, initial):
        super(GreedyAgent, self).reset(initial)
    def make_guess(self):
        return np.argmax(self.group_prob)

# Test Script
a = GreedyAgent()
a.reset((3,3,3))
guess1 = a.make_guess()
a.update_state(guess1, True)
guess2 = a.make_guess()
a.update_state(guess2, False)
print('First guess: %d | Second guess: %d | States:'%(guess1, guess2), a.states)

First guess: 0 | Second guess: 1 | States: {(1, 3, 3): Fraction(2, 5), (2, 3, 2): Fraction(3, 5)}


In [5]:
ev = deck_ev(GreedyAgent, (3,3,3))
print('Expected Correct Guesses: %.15f'%ev)

Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Users/wanqiaoxu/mambaforge/envs/esp/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/wanqiaoxu/mambaforge/envs/esp/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/wanqiaoxu/mambaforge/envs/esp/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/wanqiaoxu/mambaforge/envs/esp/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'f' on <module '__main__' (built-in)>
Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/Users/wanqiaoxu/mambaforge/envs/esp/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/wanqiaoxu/mambaforge/envs/esp/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args

KeyboardInterrupt: 

In [6]:
t0 = time.time()
initial = (3,3,3)
ev = GreedyAgent(initial=(3,3,3)).ev()
print(initial, '=> EV: %.15f (%s)'%(ev, str(ev)))
print('Took %.3f seconds'%(time.time() - t0))

(3, 3, 3) => EV: 4.241071428571429 (475/112)
Took 0.027 seconds


## Optimal Agent

Picks choices optimally by simulating itself recursively for each possible choice.

In [7]:
class OptimalAgent(Agent):
    def __init__(self, **kwargs):
        super(OptimalAgent, self).__init__(**kwargs)
        
    def reset(self, initial):
        super(OptimalAgent, self).reset(initial)
        
    def expected_correct(self, guess, memo, verbose=False):
        if self.size <= 0: return 0
        
        copy0 = OptimalAgent(initial=self.initial)
        copy0.set_composition(deepcopy(self.states))
        if copy0.update_state(guess, False): ev0 = copy0.make_guess_aux(memo)[1]
        else: ev0 = 0
        
        copy1 = OptimalAgent(initial=self.initial)
        copy1.set_composition(deepcopy(self.states))
        if copy1.update_state(guess,  True): ev1 = copy1.make_guess_aux(memo)[1] + 1
        else: ev1 = 0
            
        ev = (1 - self.group_prob[guess]) * ev0 + self.group_prob[guess] * ev1
            
        if verbose and self.size == 3:
            print(self.states)
            print('Guess:', guess, ' - ev0:', ev0, ' - ev1:', ev1, ' - ev:', ev)
            
        return ev
    
    def make_guess_aux(self, memo, verbose=False):
        serial = self.serialize()
        if serial in memo: return memo[serial]
        if sum(np.array(self.group_prob) > 0) <= 2:
            pick = np.argmax(self.group_prob)
            ret = (pick, self.expected_correct(pick, memo))
            memo[serial] = ret
            return ret
        best_ev = 0
        best_pick = 0
        for pick in range(self.group_num):
            ev = self.expected_correct(pick, memo, verbose=verbose)
            if ev > best_ev:
                best_ev = ev
                best_pick = pick
        ret = (best_pick, best_ev)
        memo[serial] = ret
        return ret
    
    def ev(self):
        memo = {}
        return self.make_guess_aux(memo)[1]
    
    def make_guess(self, verbose=False):
        memo = {}
        return self.make_guess_aux(memo, verbose=verbose)[0]

# Test Script
a = OptimalAgent()
a.reset((1,1,1))
guess1 = a.make_guess()
a.update_state(guess1, True)
guess2 = a.make_guess()
a.update_state(guess2, False)
print('First guess: %d | Second guess: %d | States:'%(guess1, guess2), a.states)

First guess: 0 | Second guess: 1 | States: {(0, 1, 0): Fraction(1, 1)}


In [8]:
t0 = time.time()
initial = (3,3,3)
ev = OptimalAgent(initial=initial).ev()
print(initial, '=> EV: %.15f (%s)'%(ev, str(ev)))
print('Took %.3f seconds'%(time.time() - t0))

(3, 3, 3) => EV: 4.255357142857143 (2383/560)
Took 0.757 seconds


## Information Agent

Picks the card maximizing the unconditioned probability of being correct, minus `gamma` times the entropy gained by making that choice.

In [34]:
class InfoAgent(Agent):
    def __init__(self, gamma=1e-5, **kwargs):
        super(InfoAgent, self).__init__(**kwargs)
        self.gamma = gamma
    def reset(self, initial):
        super(InfoAgent, self).reset(initial)
    def make_guess(self):
        best_eiv  = 0
        best_pick = 0
        for pick in range(self.group_num):
            copy0 = deepcopy(self)
            copy0.update_state(pick, False)
            ent0 = copy0.get_state_entropy()
            copy1 = deepcopy(self)
            copy1.update_state(pick, True)
            ent1 = copy1.get_state_entropy()
            entropy = (1 - self.group_prob[pick]) * ent0 + self.group_prob[pick] * ent1
            eiv = self.group_prob[pick] - self.gamma * entropy
            if eiv > best_eiv:
                best_eiv = eiv
                best_pick = pick
        return best_pick

# Test Script
a = InfoAgent()
a.reset((3,3,3))
guess1 = a.make_guess()
a.update_state(guess1, True)
guess2 = a.make_guess()
a.update_state(guess2, False)
print('First guess: %d | Second guess: %d | States:'%(guess1, guess2), a.states)

First guess: 0 | Second guess: 1 | States: {(1, 3, 3): Fraction(2, 5), (2, 3, 2): Fraction(3, 5)}


In [35]:
t0 = time.time()
initial = (3,3,3)
ev = InfoAgent(gamma=0.15, initial=(3,3,3)).ev()
print(initial, '=> EV: %.15f (%s)'%(ev, str(ev)))
print('Took %.3f seconds'%(time.time() - t0))

(3, 3, 3) => EV: 4.255357142857143 (2383/560)
Took 0.267 seconds


## Information Agent Sweeps

In [36]:
initial = (3,3,3)
opt_ev = OptimalAgent(initial=initial).ev()
greedy_ev = GreedyAgent(initial=initial).ev()

sweep_vars = np.arange(-50, 50.0001, 1) / 100.0
sweep_evs  = []
for gamma in sweep_vars:
    ev = InfoAgent(gamma=gamma, initial=initial).ev()
    print('Gamma %.1e => EV %.6f'%(gamma, ev))
    sweep_evs.append(ev)
sweep_evs = np.array(sweep_evs)

Gamma -5.0e-01 => EV 3.779762
Gamma -4.9e-01 => EV 3.780952
Gamma -4.8e-01 => EV 3.780952
Gamma -4.7e-01 => EV 3.802381
Gamma -4.6e-01 => EV 3.802381
Gamma -4.5e-01 => EV 3.802381
Gamma -4.4e-01 => EV 3.802381
Gamma -4.3e-01 => EV 3.802381
Gamma -4.2e-01 => EV 3.802381
Gamma -4.1e-01 => EV 3.802381
Gamma -4.0e-01 => EV 3.811310
Gamma -3.9e-01 => EV 3.822024
Gamma -3.8e-01 => EV 3.832738
Gamma -3.7e-01 => EV 3.832738
Gamma -3.6e-01 => EV 3.862500
Gamma -3.5e-01 => EV 3.862500
Gamma -3.4e-01 => EV 3.862500
Gamma -3.3e-01 => EV 3.862500
Gamma -3.2e-01 => EV 3.908333
Gamma -3.1e-01 => EV 3.908333
Gamma -3.0e-01 => EV 3.917857
Gamma -2.9e-01 => EV 3.917857
Gamma -2.8e-01 => EV 3.917857
Gamma -2.7e-01 => EV 3.917857
Gamma -2.6e-01 => EV 3.917857
Gamma -2.5e-01 => EV 3.917857
Gamma -2.4e-01 => EV 4.000000
Gamma -2.3e-01 => EV 4.000000
Gamma -2.2e-01 => EV 4.000000
Gamma -2.1e-01 => EV 4.054762
Gamma -2.0e-01 => EV 4.083333
Gamma -1.9e-01 => EV 4.083333
Gamma -1.8e-01 => EV 4.083333
Gamma -1.7

In [37]:
plt.figure(figsize=(20,8))
plt.plot(sweep_vars, sweep_evs, 'bo-')
plt.axhline(y=greedy_ev, color='r', linestyle='dashed')
plt.axhline(y=opt_ev, color='g', linestyle='dashed')
plt.title(r'Optimality of Information Agent vs $\gamma$', fontsize=30)
plt.xlabel(r'$\gamma$', fontsize=24)
plt.ylabel('Expected Correct', fontsize=24)
plt.grid()
plt.show()

## IDS Agent

Picks the card minimizing conditional information ratio (squared shortfall / information gain).

In [ ]:
class IDSAgent(Agent):
    def __init__(self, seed=0, **kwargs):
        super(InfoAgent, self).__init__(**kwargs)
        self.rng = np.random.default_rng(seed)
    def reset(self, initial):
        super(InfoAgent, self).reset(initial)
    def make_guess(self):
        num_type = len(type_prob)
        mutual_infos = [scipy.stats.entropy([1-p, p]) for p in type_prob]
        shortfall = 1 - type_prob
        minimum = float('inf')
        min_actions = [-1, -1]
        min_alpha = -1
        for a in range(num_type):
            func = (shortfall[a])**2 / (mutual_infos[a])
            if func < minimum:
            minimum, min_actions, min_alpha = func, [a, 0], 1
            for b in range(a, num_type):
            func = lambda x: (x * shortfall[a] + (1 - x) * shortfall[b])**2 / (x * mutual_infos[a] + (1 - x) * mutual_infos[b])
            res = scipy.optimize.minimize(func, 0, bounds=[(0, 1)])
            if res.fun < minimum:
                minimum, min_actions, min_alpha = res.fun, [a, b], res.x[0]
        if min_alpha == -1:
            return random_argmax(rng, type_prob)
        return rng.choice(min_actions, p=[min_alpha, 1 - min_alpha])

        best_eiv  = 0
        best_pick = 0
        for pick in range(self.group_num):
            copy0 = deepcopy(self)
            copy0.update_state(pick, False)
            ent0 = copy0.get_state_entropy()
            copy1 = deepcopy(self)
            copy1.update_state(pick, True)
            ent1 = copy1.get_state_entropy()
            entropy = (1 - self.group_prob[pick]) * ent0 + self.group_prob[pick] * ent1
            eiv = self.group_prob[pick] - self.gamma * entropy
            if eiv > best_eiv:
                best_eiv = eiv
                best_pick = pick
        return best_pick

# Test Script
a = IDSAgent(0)
a.reset((3,3,3))
guess1 = a.make_guess()
a.update_state(guess1, True)
guess2 = a.make_guess()
a.update_state(guess2, False)
print('First guess: %d | Second guess: %d | States:'%(guess1, guess2), a.states)

## Sweep of all initial deck compositions from (1,0,0,0) to (4,4,4,4)
This code takes a while to run (up to a day on a reasonably fast computer).

In [ ]:
initials = []
for i0 in range(4+1):
    for i1 in range(i0+1):
        for i2 in range(i1+1):
            for i3 in range(i2+1):
                initials.append((i0,i1,i2,i3))
initials = initials[1:]
print('Number of composite vectors:', len(initials))

results = {'Initial':[], 'Time':[], 'Greedy':[], 'InfoV1':[], 'Optimal':[]}
for initial in initials:
    t0 = time.time()
    
    results['Initial'].append(initial)
    print(initial)
    
    ev = GreedyAgent(initial=initial).ev()
    results['Greedy'].append(ev)
    print(' >  Greedy: %.15f (%s)'%(ev, str(ev)))
    
    ev = InfoAgent(gamma=1e-6, initial=initial).ev()
    results['InfoV1'].append(ev)
    print(' >  InfoV1: %.15f (%s)'%(ev, str(ev)))
    
    ev = OptimalAgent(initial=initial).ev()
    results['Optimal'].append(ev)
    print(' > Optimal: %.15f (%s)'%(ev, str(ev)))
    
    t0 = time.time() - t0
    results['Time'].append(t0)
    print(' > Took %.3f seconds.'%t0)
    
    with open('results.p', 'wb') as f:
        pickle.dump(results, f)
    print()

Number of composite vectors: 69
(1, 0, 0, 0)
 >  Greedy: 1.000000000000000 (1)
 >  InfoV1: 1.000000000000000 (1)
 > Optimal: 1.000000000000000 (1)
 > Took 0.002 seconds.

(1, 1, 0, 0)
 >  Greedy: 1.500000000000000 (3/2)
 >  InfoV1: 1.500000000000000 (3/2)
 > Optimal: 1.500000000000000 (3/2)
 > Took 0.004 seconds.

(1, 1, 1, 0)
 >  Greedy: 1.666666666666667 (5/3)
 >  InfoV1: 1.666666666666667 (5/3)
 > Optimal: 1.666666666666667 (5/3)
 > Took 0.028 seconds.

(1, 1, 1, 1)
 >  Greedy: 1.708333333333333 (41/24)
 >  InfoV1: 1.708333333333333 (41/24)
 > Optimal: 1.708333333333333 (41/24)
 > Took 0.144 seconds.

(2, 0, 0, 0)
 >  Greedy: 2.000000000000000 (2)
 >  InfoV1: 2.000000000000000 (2)
 > Optimal: 2.000000000000000 (2)
 > Took 0.004 seconds.

(2, 1, 0, 0)
 >  Greedy: 2.333333333333333 (7/3)
 >  InfoV1: 2.333333333333333 (7/3)
 > Optimal: 2.333333333333333 (7/3)
 > Took 0.009 seconds.

(2, 1, 1, 0)
 >  Greedy: 2.416666666666667 (29/12)
 >  InfoV1: 2.416666666666667 (29/12)
 > Optimal: 2.4

In [ ]:
import pandas as pd
with open('results.p', 'rb') as f:
    res = pickle.load(f)
df = pd.DataFrame(res)

In [ ]:
df

Greedy            InfoV1       Initial            Optimal  \
0                  1                 1  (1, 0, 0, 0)                  1   
1                3/2               3/2  (1, 1, 0, 0)                3/2   
2                5/3               5/3  (1, 1, 1, 0)                5/3   
3              41/24             41/24  (1, 1, 1, 1)              41/24   
4                  2                 2  (2, 0, 0, 0)                  2   
5                7/3               7/3  (2, 1, 0, 0)                7/3   
6              29/12             29/12  (2, 1, 1, 0)              29/12   
7              73/30             73/30  (2, 1, 1, 1)              73/30   
8               17/6              17/6  (2, 2, 0, 0)               17/6   
9               17/6              17/6  (2, 2, 1, 0)               17/6   
10           509/180           509/180  (2, 2, 1, 1)            509/180   
11            271/90            271/90  (2, 2, 2, 0)             271/90   
12            134/45            134/45  (2, 2, 2, 1)            943/315   
13             91/30             91/30  (2, 2, 2, 2)           1279/420   
14                 3                 3  (3, 0, 0, 0)                  3   
15              13/4              13/4  (3, 1, 0, 0)               13/4   
16             33/10             33/10  (3, 1, 1, 0)              33/10   
17           397/120           397/120  (3, 1, 1, 1)            397/120   
18              18/5              18/5  (3, 2, 0, 0)               18/5   
19             43/12             43/12  (3, 2, 1, 0)              43/12   
20           751/210           751/210  (3, 2, 1, 1)            751/210   
21           773/210           773/210  (3, 2, 2, 0)            773/210   
22          2043/560          2043/560  (3, 2, 2, 1)           2053/560   
23          1853/504          1853/504  (3, 2, 2, 2)          5573/1512   
24             41/10             41/10  (3, 3, 0, 0)              41/10   
25             81/20             81/20  (3, 3, 1, 0)              81/20   
26         4479/1120         4491/1120  (3, 3, 1, 1)          4503/1120   
27             81/20          1137/280  (3, 3, 2, 0)              57/14   
28         6781/1680         6781/1680  (3, 3, 2, 1)         20393/5040   
29        33673/8400         6749/1680  (3, 3, 2, 2)       101921/25200   
..               ...               ...           ...                ...   
39           467/105           467/105  (4, 2, 1, 0)            467/105   
40           533/120           533/120  (4, 2, 1, 1)            533/120   
41           947/210           947/210  (4, 2, 2, 0)            947/210   
42        16943/3780        16943/3780  (4, 2, 2, 1)           2429/540   
43       85051/18900       85051/18900  (4, 2, 2, 2)        85181/18900   
44            169/35            169/35  (4, 3, 0, 0)             169/35   
45           669/140           669/140  (4, 3, 1, 0)            669/140   
46           851/180         5963/1260  (4, 3, 1, 1)           1496/315   
47         5993/1260           857/180  (4, 3, 2, 0)               43/9   
48        14957/3150        14957/3150  (4, 3, 2, 1)         14977/3150   
49        27232/5775          1090/231  (4, 3, 2, 2)          6583/1386   
50          2923/600        20473/4200  (4, 3, 3, 0)             205/42   
51        44599/9240      223007/46200  (4, 3, 3, 1)        56137/11550   
52      443539/92400      443683/92400  (4, 3, 3, 2)           3391/700   
53       52006/10725    1456333/300300  (4, 3, 3, 3)       225307/46200   
54            373/70            373/70  (4, 4, 0, 0)             373/70   
55            473/90            473/90  (4, 4, 1, 0)             473/90   
56           519/100        10909/2100  (4, 4, 1, 1)           4687/900   
57        16393/3150          1172/225  (4, 4, 2, 0)          5483/1050   
58       89371/17325        17921/3465  (4, 4, 2, 1)         25693/4950   
59      267431/51975      267593/51975  (4, 4, 2, 2)          7679/1485   
60        30154/5775       60493/11550 